MCP - Kaggle Competition Search Agent
-------------------------------------
This script integrates Google ADK with Kaggle MCP to query Kaggle datasets
and competitions programmatically. It uses Gemini as the LLM and MCP tools
to fetch structured results (name, url, description, rating).

# 🔑 Authentication: Load Google API key 

In [1]:
import os
from kaggle_secrets import UserSecretsClient


try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please add 'GOOGLE_API_KEY' to Kaggle secrets. Details: {e}")

✅ Setup and authentication complete.


# 📦 Imports: ADK components and MCP integration

In [2]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


# 🔄 Retry config

In [3]:
# handle transient HTTP errors gracefully
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# 🔗 MCP integration with Kaggle

In [4]:
KAGGLE_MCP_TOOL = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=["-y", "mcp-remote", "https://www.kaggle.com/mcp"]
        ),
        timeout=60,  # increased timeout
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [5]:
instruction = """
You are a Kaggle Dataset and Competition Search Agent.

Responsibilities:
- Use MCP tools to search Kaggle datasets and competitions.
- Focus only on datasets relevant to the user’s query (e.g., OCR).
- Rank results by popularity and rating.

Output format:
Always return a JSON array of up to 3 items.
Each item MUST include ALL of the following keys:
{
  "name": "Dataset name (string)",
  "url": "Full Kaggle dataset URL (https://www.kaggle.com/...)",
  "description": "Short summary (1–2 sentences)",
  "rating": "Community rating or votes (string or number)"
}

Constraints:
- Do not omit any of the four keys (name, url, description, rating).
- Always include the full Kaggle dataset URL.
- Keep summaries concise and factual.
- Exclude unrelated datasets.
- If fewer than 3 datasets match, return only those available.
- If none are found, return a single JSON object: {"error": "No relevant datasets found"}.
"""

# 🤖 Define the search agent
search_agent = LlmAgent(
    name="search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction=instruction,
    tools=[KAGGLE_MCP_TOOL],
    output_key="top_3_competitions",
)

# 🏃 Runner executes the agent in-memory
runner = InMemoryRunner(agent=search_agent)
# 🔎 Query Kaggle datasets (OCR example)
response = await runner.run_debug("List 3 highly rated Kaggle datasets that contain OCR (Optical Character Recognition) data", verbose=False)


 ### Created new session: debug_session_id

User > List 3 highly rated Kaggle datasets that contain OCR (Optical Character Recognition) data


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


search_agent > ```json
[
  {
    "name": "standard OCR dataset ",
    "url": "https://www.kaggle.com/datasets/preatcher/standard-ocr-dataset",
    "description": "Optical Character Recognition Dataset containing Various Fonts and Style",
    "rating": 80
  },
  {
    "name": "Aida Calculus Math Handwriting Recognition Dataset",
    "url": "https://www.kaggle.com/datasets/aidapearson/ocr-data",
    "description": "Synthetic handwritten calculus math expressions for recognition and OCR tasks",
    "rating": 74
  },
  {
    "name": "OCR Receipts Text Detection - retail dataset",
    "url": "https://www.kaggle.com/datasets/trainingdatapro/ocr-receipts-text-detection",
    "description": "Photos of the receipts and text detection - ocr dataset",
    "rating": 70
  }
]
```
